In [1]:
#Ativando ambiente virtual com python 3.7
!conda activate detectron2

In [2]:
!nvidia-smi

Mon Jan 24 22:56:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   44C    P8     2W /  N/A |    489MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Instala o OpenCV
#!pip install -q opencv-python==4.2.0.34

In [ ]:
# Instala o OpenCV Headless
# !pip install -q opencv-python-headless==4.2.0.34

In [ ]:
# Instala API COCO (formato dos dados para treinar os modelos)
#!pip install -q 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
# Instala PyTorch com suporte a plataforma CUDA (somente para computadores com GPU)
#!pip install -q -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 

In [ ]:
# Instala pacote para suporte a C++
#!pip install -q cython 

In [ ]:
# Instala pyyaml
#!pip install -q --ignore-installed pyyaml==5.1

As duas células abaixo devem ser executadas somente na primeira vez que executar este Jupyter na máquina. 

**Isso deve ser feito somente na primeira execução deste notebook**.

In [ ]:
# Clone do repositório do Detectron2
# Seu computador pode não ter o Git instalado. Acesse este link e instale de acordo com seu sistema operacional:
# https://git-scm.com/book/en/v2/Getting-Started-Installing-Git
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo

In [ ]:
# Instala o Detectron2
# Somente na primeira vez que executar este Jupyter Notebook
#!pip install -q -e detectron2_repo

>Uma vez instaladas as dependências acima, você precisará reiniciar o Jupyter Notebook no terminal. Salve este notebook, feche-o, pare o Jupyter Notebook, inicie novamente e continue o trabalho.

In [ ]:
# Imports gerais
import os
import cv2
import glob
import json
import ntpath
import random
import urllib
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import PIL.Image as Image

# Torch
import torch
import torchvision

# Detectron2
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
setup_logger()

# Gráficos e Imagens
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Formatação das imagens
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

# Seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Conjunto de Dados

Nosso conjunto de dados é fornecido por [Dataturks](https://dataturks.com/), e está disponível no [Kaggle](https://www.kaggle.com/dataturks/face-detection-in-images). 

O dataset:

> Temos cerca de 500 imagens com 1100 faces marcadas manualmente através da caixa delimitadora.

Fiz o download do arquivo JSON que contém as anotações. Vamos carregá-lo:

In [ ]:
# Carrega o arquivo JSON
faces_df = pd.read_json('dicionarios/face_detection.json', lines = True)

In [ ]:
# Visualiza o conteúdo
faces_df.head()

Cada linha contém uma anotação de face única. Observe que várias linhas podem apontar para uma única imagem (por exemplo, várias faces por imagem).

## Pré-Processamento de Dados

O conjunto de dados contém apenas URLs e anotações de imagem. Teremos que baixar as imagens. Também normalizamos as anotações, para que seja mais fácil usá-las com o Detectron2 posteriormente.

In [ ]:
# Cria o diretório para gravar as imagens
os.makedirs("faces", exist_ok = True)

In [ ]:
# Lista para receber os dados
dataset = []

In [ ]:
# Loop
for index, row in tqdm(faces_df.iterrows(), total = faces_df.shape[0]):
    
    # Define a url de cada imagem
    img = urllib.request.urlopen(row["content"])
    
    # Abre a imagem
    img = Image.open(img)
    
    # Converte para RGB (3 cores)
    img = img.convert('RGB')

    # Define o nome da imagem
    image_name = f'face_{index}.jpeg'

    # Salva a imagem emdisco
    img.save(f'faces/{image_name}', "JPEG")
    
    # Obtém a anotação (label da imagem)
    annotations = row['annotation']
    
    # Loop pela anotação de cada imagem
    for an in annotations:

        # Dicionário
        data = {}

        # Largura, altura e pontos da imagem
        width = an['imageWidth']
        height = an['imageHeight']
        points = an['points']

        # Nome, largura e altura da imagem
        data['file_name'] = image_name
        data['width'] = width
        data['height'] = height

        # Coordenadas da bounding box (caixa delimitadora) de cada face na imagem
        data["x_min"] = int(round(points[0]["x"] * width))
        data["y_min"] = int(round(points[0]["y"] * height))
        data["x_max"] = int(round(points[1]["x"] * width))
        data["y_max"] = int(round(points[1]["y"] * height))

        # Nome da classe
        data['class_name'] = 'face'

        # Grava o resultado na lista
        dataset.append(data)

Vamos colocar os dados em um dataframe.

In [ ]:
# Converte a lista em um dataframe
df = pd.DataFrame(dataset)

In [ ]:
# Visualiza
df.head()

In [ ]:
# Shape
print(df.file_name.unique().shape[0], df.shape[0])

Temos um total de 409 imagens e 1132 anotações. Vamos salvá-los no disco (para que você possa reutilizá-los).

In [ ]:
# Gravando as anotações
df.to_csv('dicionarios/annotations.csv', header = True, index = None)

### Análise Exploratória

Vamos ver alguns exemplos de dados anotados. Usaremos o OpenCV para carregar uma imagem, adicionar as caixas delimitadoras e redimensioná-la. Definiremos uma função auxiliar para fazer tudo isso.

In [ ]:
# Função visualizar as imagens a partir das anotações
def annotate_image(annotations, resize = True):
    
    # Nome do arquivo
    file_name = annotations.file_name.to_numpy()[0]
    
    # Leitura da imagem com OpenCV
    img = cv2.cvtColor(cv2.imread(f'faces/{file_name}'), cv2.COLOR_BGR2RGB)

    # Busca as anotações 
    for i, a in annotations.iterrows():    
        cv2.rectangle(img, (a.x_min, a.y_min), (a.x_max, a.y_max), (0, 255, 0), 2)

    # Redimensiona a imagem se necessário
    if not resize:
        return img

    # Retroa a imagem
    return cv2.resize(img, (384, 384), interpolation = cv2.INTER_AREA)

In [ ]:
# Visualizando uma imagem com anotação
img_df = df[df.file_name == df.file_name.unique()[5]]
img = annotate_image(img_df, resize = False)
plt.imshow(img)
plt.axis('off')

In [ ]:
# Visualizando uma imagem com anotação
img_df = df[df.file_name == df.file_name.unique()[9]]
img = annotate_image(img_df, resize = False)
plt.imshow(img)
plt.axis('off')

Essas são boas, as anotações são claramente visíveis. Podemos usar o torchvision para criar uma grade de imagens. Observe que as imagens estão em vários tamanhos, então vamos redimensioná-las.

In [ ]:
# Obtendo amostras de imagens
sample_images = [annotate_image(df[df.file_name == f]) for f in df.file_name.unique()[:10]]
sample_images = torch.as_tensor(sample_images)

In [ ]:
# Shape do tensor
sample_images.shape

In [ ]:
# Precisamos ajustar o shape
sample_images = sample_images.permute(0, 3, 1, 2)

In [ ]:
# Shape
sample_images.shape

In [ ]:
# Plot de várias imagens em grid
plt.figure(figsize = (24, 12))
grid_img = torchvision.utils.make_grid(sample_images, nrow = 5)
plt.imshow(grid_img.permute(1, 2, 0))
plt.axis('off')

Você pode ver claramente que algumas anotações estão ausentes (coluna 4). Teremos que lidar com isso de alguma forma.

## Preparando os Dados Para o Modelo

In [ ]:
# Carregamos as anotações
df = pd.read_csv('dicionarios/annotations.csv')

In [ ]:
# Diretório das imagens
IMAGES_PATH = f'faces'

In [ ]:
# Retorna imagens únicas
unique_files = df.file_name.unique()

In [ ]:
# Prepara os dados de treino e de teste
# O train_test_split clássico não funcionaria aqui, porque queremos uma divisão entre os nomes dos arquivos.
train_files = set(np.random.choice(unique_files, int(len(unique_files) * 0.95), replace = False))
train_df = df[df.file_name.isin(train_files)]
test_df = df[~df.file_name.isin(train_files)]

In [ ]:
# Dados de treino
train_df.head()

In [ ]:
# Obtém as classes
classes = df.class_name.unique().tolist()

In [ ]:
# Temos como classe de saída apenas a face na imagem
classes

Em seguida, escreveremos uma função que converte nosso conjunto de dados em um formato usado pelo Detectron2, o COCO.

In [ ]:
# Função para colocar as imagens em formato COCO
def create_dataset_dicts(df, classes):
    
    # Lista para os dicionários de classe
    dataset_dicts = []
        
    # Loop por cada imagem
    for image_id, img_name in enumerate(df.file_name.unique()):

        # Dicionário
        record = {}

        # Imagem
        image_df = df[df.file_name == img_name]

        # Caminho
        file_path = f'{IMAGES_PATH}/{img_name}'
        
        # Dados da imagem
        record["file_name"] = file_path
        record["image_id"] = image_id
        record["height"] = int(image_df.iloc[0].height)
        record["width"] = int(image_df.iloc[0].width)

        # Objetos
        objs = []
    
        # Loop
        for _, row in image_df.iterrows():

            # Coordenadas
            xmin = int(row.x_min)
            ymin = int(row.y_min)
            xmax = int(row.x_max)
            ymax = int(row.y_max)

            poly = [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]
            poly = list(itertools.chain.from_iterable(poly))

            obj = {"bbox": [xmin, ymin, xmax, ymax], 
                   "bbox_mode": BoxMode.XYXY_ABS, 
                   "segmentation": [poly], 
                   "category_id": classes.index(row.class_name), 
                   "iscrowd": 0}
    
            objs.append(obj)

    record["annotations"] = objs
    dataset_dicts.append(record)
    return dataset_dicts

Convertemos cada linha de anotação em um único registro com uma lista de anotações. Você também pode perceber que estamos criando um polígono exatamente da mesma forma que a caixa delimitadora. Isso é necessário para os modelos de segmentação de imagens no Detectron2.

Você precisará registrar seu conjunto de dados nos catálogos de conjunto de dados e metadados. Faremos isso na célula abaixo.

In [ ]:
# Registrando o conjunto de dados
for d in ["train", "val"]:
    DatasetCatalog.register("faces_" + d, lambda d=d: create_dataset_dicts(train_df if d == "train" else test_df, classes))
    MetadataCatalog.get("faces_" + d).set(thing_classes=classes)

In [ ]:
# Gerando os metadados
statement_metadata = MetadataCatalog.get("faces_train")

Infelizmente, o avaliador para o conjunto de testes não é incluído por padrão. Podemos resolver isso facilmente, escrevendo nossa própria classe.

In [ ]:
# Classe paar trenar e avaliar o modelo
class treinaModelo(DefaultTrainer):
  
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder = None):

        if output_folder is None:
            os.makedirs("eval", exist_ok = True)
            output_folder = "eval"

        return COCOEvaluator(dataset_name, cfg, False, output_folder)

Os resultados da avaliação serão armazenados na pasta `eval` se nenhuma pasta for fornecida.

O ajuste fino de um modelo Detectron2 não é nada como escrever código PyTorch. Carregaremos um arquivo de configuração, alteraremos alguns valores e iniciaremos o processo de treinamento. 

Usaremos o modelo pré-treinado Mask R-CNN X101-FPN. 

Esse modelo foi pré-treinado no dataset [COCO dataset](http://cocodataset.org/#home) e alcança um desempenho muito bom. A desvantagem é que é lento para treinar.

Vamos carregar o arquivo de configuração e os pesos do modelo pré-treinado.

In [ ]:
# Inicia o arquivo de configuração
cfg = get_cfg()

# Carrega o arquivo de configuração
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))

# Carrega os pesos do modelo pré-treinado
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

In [ ]:
# Especificamos os conjuntos de dados (nós os registramos) que usaremos para treinamento e avaliação
cfg.DATASETS.TRAIN = ("faces_train",)
cfg.DATASETS.TEST = ("faces_val",)
cfg.DATALOADER.NUM_WORKERS = 4

In [ ]:
# E para o otimizador, definiremos alguns valores. Fique à vontade para alterar esses valores e re-treinar o modelo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.TEST.EVAL_PERIOD = 500

Os primeiros 3 parâmetros são padrões (tamanho do lote, número máximo de iterações e taxa de aprendizado).

Mas temos ainda:

- `WARMUP_ITERS` - a taxa de aprendizado começa em 0 e vai para a predefinida para este número de iterações
- `STEPS` - os pontos de verificação (número de iterações) nos quais a taxa de aprendizado será reduzida em `GAMMA`

Por fim, especificaremos o número de classes e o período em que avaliaremos no conjunto de teste.

In [ ]:
# Checamos se o diretório de saída existe
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

E então treinamos o modelo.

In [ ]:
# Carregamos as configurações
trainer = treinaModelo(cfg) 

In [ ]:
# Vamos iniciar o treinamento do zero e não de onde parou
trainer.resume_or_load(resume = False)

In [ ]:
# Treinamento - Dura 30 minutos no Titan
trainer.train()

## Carregando o Modelo

In [ ]:
# Diretório e nome do modelo
cfg.MODEL.WEIGHTS = os.path.join("output", "model_final.pth")

# Define o threshold de teste para o modelo
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75

# Salva o preditor
preditor = DefaultPredictor(cfg)

## Avaliando o Modelo

A avaliação de modelos de detecção de objetos é um pouco diferente quando comparada à avaliação de modelos padrão de classificação ou regressão.

A principal métrica que você precisa conhecer é IoU (interseção sobre união). Ele mede a sobreposição entre dois limites - o predito e a verdade fundamental. Pode obter valores entre 0 e 1.

$$\text{IoU}=\frac{\text{area of overlap}}{\text{area of union}}$$

Usando IoU, pode-se definir um limite (por exemplo > 0,5) para classificar se uma previsão é um verdadeiro positivo (TP) ou um falso positivo (FP).

Agora você pode calcular a precisão média (AP) tomando a área sob a curva de precisão e recuperação. Isso deve fornecer uma compreensão prática de como os modelos de detecção de objetos são avaliados.

In [ ]:
# Resultado da avaliação
evaluator = COCOEvaluator("faces_val", cfg, False, output_dir = "./output/")
val_loader = build_detection_test_loader(cfg, "faces_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

### Detecção Facial

Vamos ver o modelo funciona e fazer detecção facial em novas imagens.

In [ ]:
# Define o diretório
os.makedirs("annotated_results", exist_ok = True)

In [ ]:
# Obtém as imagens
test_image_paths = test_df.file_name.unique()

In [ ]:
# Aqui fazemos detecção para cada imagem de teste
for teste_image in test_image_paths:
    file_path = f'{IMAGES_PATH}/{teste_image}'
    im = cv2.imread(file_path)
    outputs = preditor(im)
    v = Visualizer(im[:, :, ::-1], metadata = statement_metadata, scale = 1., instance_mode = ColorMode.IMAGE)
    instances = outputs["instances"].to("cpu")
    instances.remove('pred_masks')
    v = v.draw_instance_predictions(instances)
    result = v.get_image()[:, :, ::-1]
    file_name = ntpath.basename(teste_image)
    write_res = cv2.imwrite(f'annotated_results/{file_name}', result)

Vamos conferir.

In [ ]:
annotated_images = [f'annotated_results/{f}' for f in test_df.file_name.unique()]

In [ ]:
# Detectando faces em imagens
img = cv2.cvtColor(cv2.imread(annotated_images[5]), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')

In [ ]:
# Detectando faces em imagens
img = cv2.cvtColor(cv2.imread(annotated_images[3]), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')

In [ ]:
# Detectando faces em imagens
img = cv2.cvtColor(cv2.imread(annotated_images[11]), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')

In [ ]:
# Detectando faces em imagens
img = cv2.cvtColor(cv2.imread(annotated_images[9]), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')

O modelo funcionou bem! Perceba que alguns erros ocorrram, o que é normal. Experimente ajustar os hiperparâmetros e treinar o modelo por mais tempo.

## Conclusão

Parabéns! Agora você sabe como usar o Detectron2 com PyTorch para detecção de objetos! Usar modelos pré-treinados pode ser uma arma poderosa para construir aplicações de Visão Computacional.

# Fim